<img src="https://raw.githubusercontent.com/ryanedw/COMPSS-202-SU24/main/Images/UCB-macss.jpg" width="120" align="right"/>
<h1>COMPSS 202 Guided Project 3</h1>

<h2>An early RCT with a small sample. Big enough to reject the null?</h2>

Feel free to work together in teams of 2-4 students. Please answer each question below in your own original and complete sentences. Questions appear in <b><font color="red">bold red</font></b>. Be parsimonious: brief while also hitting the main points. Students earn scores on this project for complete answers, which can be either subjectively correct or incorrect.

<b>This exercise is intended to blend familiarity with confusion.</b> Some questions have clear answers. Others have many answers, some of which are probably better answers than others, but all might still be correct in one way or another.

To begin, please run the cells below to load up the libraries necessary to access data in Google Sheets. Best practices include running the cells in order.

In [ ]:
install.packages("googlesheets4")
library(googlesheets4)
gs4_deauth()

<h2>Streptomycin and tuberculosis</h2>

As recounted by [Metcalfe (2011)](https://www-bmj-com.libproxy.berkeley.edu/content/317/7167/1248.3.full.print), Geoffrey Marshall (1887-1982) and the Medical Research Council (MRC) of the UK conducted what some have called the first "randomized curative trial" of streptomycin on patients with pulmonary tuberculosis between 1946 and 1948. The earliest clinical trial is usually identified as the efforts by [James Lind](https://en.wikipedia.org/wiki/James_Lind) to understand treatments for sailors in the Royal Navy afflicted with scurvy. Modern [randomized controlled trials (RCTs)](https://en.wikipedia.org/wiki/Randomized_controlled_trial) randomize participants into control and treatment groups; usually are double-blinded, meaning that both participants and investigators do not actively know which group a participant is in; and usually give placebos to the control group.

[Tuberculosis](https://en.wikipedia.org/wiki/Tuberculosis) is a deadly disease that still plagues less developed regions of the world. The [World Health Organization (WHO)](https://www.who.int/health-topics/tuberculosis#tab=tab_1) rates TB as the "world's top infectious killer." Following the earlier discovery of penicillin, [streptomycin](https://en.wikipedia.org/wiki/Streptomycin) was first isolated in soil from New Jersey in 1943. According to Metcalfe (2011), high costs of producing streptomycin motivated the British government to conduct a randomized controlled trial to examine its efficacy.

The MRC published the results in the <i>BMJ</i> in 1948, and they were [reprinted in BMJ in 1998](https://www-bmj-com.libproxy.berkeley.edu/content/317/7167/1248.3).

The basic story in this RCT is that 107 similar patients with tuberculosis were randomized into a treatment group of 55 patients receiving streptomycin in addition to bed rest, and a control group of 52 patients that received bed rest alone. After 6 months, 4 members of the treatment group and 14 members of the control group had died. All the relevant data for a simple analysis are in the table below:

| Time     | Control | Treatment |
|:----------|---------|-----------|
| start    | 52      | 55        |
| 6 months | 38      | 51        |
| deaths   | 14      | 4         |


In addition to a simple analysis using some symbolic math, we will also see how `lm()` produces ordinary least squares estimates that are very similar, kind of like using that big ladder in "Chutes and Ladders" to quickly jump ahead! Below is a simple data frame that shows all 107 patients along the rows, with two binary variables registering 0's and 1's for treatment status, `treat`, and whether they had died after 6 months, `died`.

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1QQhirV9mGwN9M5gl3gYIc-5E2RExgW6XxSJo0D9cvhI/edit?usp=sharing"

strep <- read_sheet(sheet_url,
                    range = "A1:C108")
head(strep)

The basic story is that we would like to know whether the observed difference in 6-month death rates is <i>statistically significant</i> or not. Let's write the death rate in group $g = t, c$ as the following: $p_g = d_g / n_g$, where $d_g$ measures the deaths and $n_g$ measures the initial count of patients. 

As shown in the code below, the study results show that
* $p_t = 4/55 \approx 0.073$  or about 7.3 percent
* $p_c = 14/52 \approx 0.269$ or about 26.9 percent
  
And our <b>null hypothesis</b> is 

$$
H_0: p_t - p_c = 0
$$ 

That is, we are testing whether the difference between the death rates is zero or null. Certainly the point estimates imply a big difference: $p_t - p_c \approx -0.197$, but the total sample is only 107 individuals. What are the chances that other samples might have produced estimates of $p_t - p_c$ that are at or above zero?

<h2>1. Symbolic math and the $SE$ of a difference in proportions</h2>

As we have seen in COMPSS 202, convention is that the standard error of a sample proportion $p_g$ is given by

$$
SE[p_g] = \sqrt{ \frac{ p_g (1 - p_g)}{n_g} }
$$

where $n_g$ is the sample size. Here, we would like to know the standard error of the <i>difference</i> of two sample proportions: $p_t - p_c$. The sample proportions are both random variables. To proceed, recall the formula for the variance of a linear combination of random variables:

$$
Var[aX + bY] = a^2 \ Var[X] + b^2 \ Var[Y] + 2ab \ Cov[X,Y]
$$

In our case, $a = 1$ and $b = -1$, and there is zero covariance between the sample proportions drawn from two separate samples. (The same would not be true if we examine an $X$ and a $Y$ measured on the same individual, as was the case with subjective and objective height.) Under these conditions, this reduces to:

$$
Var[p_t - p_c] = Var[p_t] + Var[p_c] = (SE[p_t])^2 + (SE[p_c])^2
$$

The standard error of a sample statistic is the square root of its variance:

$$
SE[p_t - p_c] = \sqrt{ (SE[p_t])^2 + (SE[p_c])^2 } = 
\sqrt{ \frac{ p_t (1 - p_t)}{n_t} + \frac{ p_c (1 - p_c)}{n_c} }
$$

The math may make your head swim, but the last part on the right-hand side above is the finish line. Let's substitute the values to help make it clearer:

$$
SE[p_t - p_c]
= \sqrt{ \frac{ 0.073 (1 - 0.073)}{55} + \frac{ 0.269 (1 - 0.269)}{52} }
$$

Now let's dive in and run these calculations. First, the proportions and the sample sizes:

In [ ]:
pt = mean(subset(strep,treat == 1)$died)
pt

# run as a check:
4/55

nt = nrow(subset(strep,treat == 1))
nt

In [ ]:
pc = mean(subset(strep,treat == 0)$died)
pc

# run as a check:
14/52

nc = nrow(subset(strep,treat == 0))
nc

Let's define the difference $p_t - p_c \equiv \beta$. I think __R__ uses `beta` for a function, so let's call it `betap` instead:

In [ ]:
betap = pt - pc
betap

That's large. Is it large enough given the small sample size to reject the null? Here are the standard errors of the sample proportions:

In [ ]:
se_pt = sqrt( pt*(1-pt)/nt )
se_pt

In [ ]:
se_pc = sqrt( pc*(1-pc)/nc )
se_pc

And finally, here is $SE[\beta] = SE[p_t - p_c]$, which equals the square root of the variance of $\beta$:

In [ ]:
se_betap = sqrt( se_pt^2 + se_pc^2 )
se_betap

<h3>A 95% confidence interval</h3>

In COMPSS 202, we have seen how random sampling and the Central Limit Theorem results in sample statistics that are approximately normal, lying within roughly ± 2 of their standard errors of the true population statistic.

In [ ]:
betap_95lo = betap - 2*se_betap
betap_95hi = betap + 2*se_betap

betap_95lo
betap_95hi

<b><font color="red">1. Does the 95% confidence interval around $\beta$ include zero? If it does, then we would fail to reject the null hypothesis that the true population $\beta = p_t - p_c = 0$. Or do we reject the null? Pick the correct answer and explain.</font></b>

---

<h2>2. A more precise tool? <span style="font-family:Courier; ">prop.test()</span></h2>

What we've done here is rudimentary but perfectly adequate. COMPSS 202 is designed to help make arcane things intuitive, and there also are more direct, possibly more precise ways of tackling arcane things.

Our rule of about ± 2 $SE$'s is an approximation, and there also are different ways to think about the standard error of a difference in proportions. If we're concerned about precision, there might be other estimation methods to explore. If instead we're concerned about speed, it might be useful to examine how ordinary least squares can help us. (Stay tuned.)

There are many methods to construct statistical tests. If you ask ChatGPT how to test the equality of two proportions in __R__, it may suggest that you use `prop.test()`. [This entry on StackExchange](https://stats.stackexchange.com/questions/2391/what-is-the-relationship-between-a-chi-squared-test-and-test-of-equal-proportion) suggests that `prop.test()` calculates the standard error a little differently:

$$
SE[p_t - p_c]
=
\sqrt{
\frac{d_t + d_c}{n_t + n_c}
\left(
1 - 
\frac{d_t + d_c}{n_t + n_c}
\right)
\left( 
\frac{1}{n_t} + \frac{1}{n_c}
\right)
}
$$

which also matches [this guidance from STAT 800 at Penn State](https://online.stat.psu.edu/stat800/lesson/5/5.5).

That standard error, call it `se_beta`, turns out to be:

In [ ]:
dt = 4
dc = 14

p = (dt + dc)/(nt + nc)

se_beta = sqrt( p*(1-p)*(1/nt + 1/nc) )
se_beta

<b><font color="red">2(a). How does this standard error compare to the one you calculated above? Will it widen or narrow the 95% confidence interval, or will it leave it unaffected? Briefly discuss.</font></b>

Formally using `prop.test()` here looks like this. As you can see, it's not obvious what the routine is actually doing! The hope is that because you've seen COMPSS 202, you'll have some context and intuition with which to judge the output.

In [ ]:
deaths = c(4, 14)
sample_sizes = c(55, 52)

test_equal_rates = prop.test(deaths, sample_sizes)
test_equal_rates

<b><font color="red">2(b). How does the 95% confidence interval generated by `prop.test()` compare to the one you calculated above? Qualitatively speaking, does anything change about your rejection or failure to reject the null hypothesis? Briefly discuss.</font></b>

---

<h2>3. A "big ladder" for most uses: <span style="font-family:Courier; ">lm()</span></h2>

There are at least two things in applied statistics that you actually can count on:

1. Any two statisticians will likely disagree on the precise format of the test
2. Ordinary least squares (OLS) is a "big ladder" that will usually get you the right answer, at least approximately

(The important caveat is that we have said nothing yet about <b>causality</b>. But in a genuine RCT, with randomization, double-blinding, and a placebo, causality running from the treatment into the outcome is strong.)

The code below uses `lm()` to run OLS on this equation:

$$
died_i = \alpha + \beta \ treat_i + \epsilon_i
$$

where $i$ indexes the 107 patients in the study; $died_i$ is a binary indicator of the patient's having died within six months; $treat_i$ is a binary indicator of belonging to the randomly selected treatment group; and $\epsilon_i$ is a white-noise error term.

One of the tricks is to "stack," "pool," or "append" the data into a single data frame. I did this already, but in your own analyses of other data, you might need to combine subsets. In __R__, you could use `rbind()` for "row bind" to combine two data frames in this way, adding observations (rows) from one dataset to another.

In [ ]:
reg1 <- lm(died ~ treat, 
           data = strep)
summary(reg1)
confint(reg1)

<b><font color="red">3. How does the 95% confidence interval generated by OLS, shown on the line directly above, compare to the two others that you calculated using methods in sections 1 and 2 above? Qualitatively speaking, does anything change about your rejection or failure to reject the null hypothesis? Briefly discuss.</font></b>

<h3>Use that big ladder, but humbly!</h3> 

Coming up in your MaCSS training, you will see many other perspectives about appropriate modeling techniques. Economists and perhaps others may occupy one end of a spectrum of opinions about the appropriateness of using OLS in cases where the $Y$ variable is discrete, as it is here. In modern scientific research on RCTs that test the response of illness and death to treatments, one will probably never see OLS. There are many other estimation methods more suited to the application.

But OLS is a parsimonious building block in statistical thinking, and it is easy and quick. Use it often in social science, and use it humbly in mixed company.

---

<h2>4. How close was failure?</h2>

Let's use the parsimonious `prop.test()` code below to probe how close the MRC's study literally was to failing; that is, failing to reject the null hypothesis. The 1948 writeup reveals that 6 members of the treatment group experienced "Considerable deterioration" by six months, as did 6 members of the control group. Suppose more of those 6 in the treatment group had died.

<b><font color="red">4(a). What is the minimum number of deaths in the <i>treatment group</i> that would have widened the 95% confidence interval sufficiently and caused a failure to reject the null hypothesis? Find the answer by changing the `4` that you see in the code below, and then briefly write about what you found.</font></b>

In [ ]:
deaths_f = c(6, 14)
sample_sizes_f = c(55, 52)

test_equal_rates_f = prop.test(deaths_f, sample_sizes_f)
test_equal_rates_f

<b><font color="red">4(b). The MRC's streptomycin trial was single-blinded, meaning that physicians knew which patients had been treated. In light of your finding immediately above, does this raise any concerns in your mind about the original results of the study?</font></b>

---

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>